**Qiime2 codes for processing zooplankton data from HN00126595_report**

Script here is based on Kristin Yoshimura's QIIME2 tutorial 
https://www.youtube.com/watch?v=cEbYCzTzQr8 and
https://docs.qiime2.org/2020.6/ and
https://chmi-sops.github.io/mydoc_qiime2.html

## 1. Prepare all requires input files for the analysis

#### 1.1 Raw sequence data files (.fastq.gz)
We first prepare data in 'seq-raw-data/' folder under this 'qiime2-zooplankton1/' folder

There are 9 files in total for this analysis. All musted be named according to QIIME2 casava 1.8 demultiplex (paired-end) format. These files were taken from 'HN00126595_report' Macrogen report and renamed according to 
qiime2 format. 

For example,
G1_1_L001_R1_001.fastq.gz
G1_1_L001_R2_001.fastq.gz
G2_2_L001_R1_001.fastq.gz
G2_2_L001_R2_001.fastq.gz

Execute 'tools import' to generate 'demux-paired-end.qza'

Note that somehow, to use QIIME2 casava 1.8 demultiplex format on Macrogen raw data, the last term for all sequence files have to be 001. 

#### 1.2 Metadata file (.tsv)
Metdata file 'zooplankton1MetaData.tsv' is put in our root 'qiime2-zooplankton1/' folder
This file has info about(dummy) barcode and sample-source: Chlorella, Moina or Branchinella (Thai fairy shrimp)

#### 1.3 Pre-trained (.qza)
Here, we will use pre-trained classifier for taxonomic assignment 
We use the following classifer for now from Green Gene

'gg-13-8-99-nb-classifier.qza'

## 2. Import raw sequence data into QIIME2

In [14]:
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path seq-raw-data \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux-paired-end.qza

Imported seq-raw-data as CasavaOneEightSingleLanePerSampleDirFmt to demux-paired-end.qza


**Now,let's visualizing demux-paired-end.qza using 'demux summarize' and 'tools view'**

In [ ]:
!qiime demux summarize \
--i-data demux-paired-end.qza \
--o-visualization demux-paired-end

In [4]:
!qiime tools view demux-paired-end.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

## 3. QC and Denoising data .. (and merge pair-ended seq together)

QC and denoising data in 'demux-paired-end.qza'  **this step take about 15 minutes**

After this step, we will have 3 output files in our 'qiime2-zooplankton1/' folder
1. table.qza -- feature table.. showing how many different otus we have in each sample
2. rep-seqs.qza -- representative sequence from each otu
3. denoising-stats -- info on the number of read count we have on each stage of denoising and pairing

Note that deoising could fail if: 
a) we truncated sequence too much to the point that pair-end reads cannot be merged --> need to reduce truncation
b) we did not allocate enough RAM --> need to close virsualbox with saving, re-open and go to setting > system to allocate more memory

In [1]:
!qiime dada2 denoise-paired \
--i-demultiplexed-seqs demux-paired-end.qza \
--p-trim-left-f 10 \
--p-trim-left-r 10 \
--p-trunc-len-f 290 \
--p-trunc-len-r 260 \
--o-table table \
--o-representative-sequences rep-seqs \
--o-denoising-stats denoising-stats \
--verbose

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_paired.R /tmp/tmphqg4nr3f/forward /tmp/tmphqg4nr3f/reverse /tmp/tmphqg4nr3f/output.tsv.biom /tmp/tmphqg4nr3f/track.tsv /tmp/tmphqg4nr3f/filt_f /tmp/tmphqg4nr3f/filt_r 290 260 10 10 2.0 2.0 2 independent consensus 1.0 1 1000000

R version 3.5.1 (2018-07-02) 
Loading required package: Rcpp
DADA2: 1.10.0 / Rcpp: 1.0.4.6 / RcppParallel: 5.0.0 
1) Filtering .........
2) Learning Error Rates
90254920 total bases in 322339 reads from 9 samples will be used for learning the error rates.
80584750 total bases in 322339 reads from 9 samples will be used for learning the error rates.
3) Denoise samples .........
.........
4) Remove chimeras (method = consensus)
6) Write output
Saved FeatureTable[Frequency] to: table.qza
Saved FeatureData[Sequence] to

Now, let's look at our denoising statistics

In [2]:
# make .qzv for denoising stats
!qiime metadata tabulate \
--m-input-file denoising-stats.qza \
--o-visualization denoising-stats

Saved Visualization to: denoising-stats.qzv


In [8]:
# visualizing .qzv file
!qiime tools view denoising-stats.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

## 4. Build a phylogenetic tree

Here, we use representative sequence in 'rep-seqs.qza' to build a phylogenetic tree.
rep-seqs.qza -> aligned-rep-seqs.qz -> masked-aligned-rep-seqs.qza -> unrooted-tree.qza -> rooted-tree.qza

In [4]:
# 4.1 sequence alignment (rep-seqs.qza -> aligned-rep-seqs.qza)
!qiime alignment mafft \
--i-sequences rep-seqs.qza \
--o-alignment aligned-rep-seqs

Saved FeatureData[AlignedSequence] to: aligned-rep-seqs.qza


In [5]:
# 4.2 mask out non-informative regions of the sequence (aligned-rep-seqs.qza -> masked-aligned-rep-seqs.qza)
!qiime alignment mask \
--i-alignment aligned-rep-seqs.qza \
--o-masked-alignment masked-aligned-rep-seqs.qza

Saved FeatureData[AlignedSequence] to: masked-aligned-rep-seqs.qza


In [6]:
# 4.3 create an unrooted tree (masked-aligned-rep-seqs.qza -> unrooted-tree.qza)
!qiime phylogeny fasttree \
--i-alignment masked-aligned-rep-seqs.qza \
--o-tree unrooted-tree

Saved Phylogeny[Unrooted] to: unrooted-tree.qza


In [7]:
# 4.4 create a rooted tree from unrooted tree (unrooted-tree.qza -> rooted-tree.qza)
!qiime phylogeny midpoint-root \
--i-tree unrooted-tree.qza \
--o-rooted-tree rooted-tree

Saved Phylogeny[Rooted] to: rooted-tree.qza


## 5. Diversity Analysis

Alpha and beta diversity analysis using data from the following files
- zooplankton1MetaData.qza
- table.qza
- rooted-tree.qza

In [14]:
# perform diversity analysis and save analysis output files (qza and qzv in core-metrics-results)
!qiime diversity core-metrics-phylogenetic \
--i-phylogeny rooted-tree.qza \
--i-table table.qza \
--p-sampling-depth 20000 \
--m-metadata-file zooplankton1MetaData.tsv \
--output-dir core-metrics-results

Saved FeatureTable[Frequency] to: core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/jaccard_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/bray_curtis_pcoa_re

In [15]:
# Look at alpha diversity
!qiime diversity alpha-group-significance \
--i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
--m-metadata-file zooplankton1MetaData.tsv \
--o-visualization core-metrics-results/faith-pd-group-significance

Saved Visualization to: core-metrics-results/faith-pd-group-significance.qzv


In [16]:
# look at a boxplot of alpha diversity
!qiime tools view core-metrics-results/faith-pd-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [17]:
# look at beta diveristy (emperor plot 3D)
!qiime tools view core-metrics-results/unweighted_unifrac_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [18]:
# alpha rarefaction plots
!qiime diversity alpha-rarefaction \
--i-table core-metrics-results/rarefied_table.qza \
--p-max-depth 20000 \
--m-metadata-file zooplankton1MetaData.tsv \
--p-steps 25 \
--o-visualization alpha-rarefaction

Saved Visualization to: alpha-rarefaction.qzv


In [19]:
# visualing alpha rarefaction plots
!qiime tools view alpha-rarefaction.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

## 6. Taxonomic Assignment 

In [ ]:
Use pre-trained classifier to assignt taxonomy to each represetative sequence of our data. 

In [20]:
# use classifer to classify representative sequence --> make taxonomy.qza file
!qiime feature-classifier classify-sklearn \
--i-classifier gg-13-8-99-nb-classifier.qza \
--i-reads rep-seqs.qza \
--o-classification taxonomy

Saved FeatureData[Taxonomy] to: taxonomy.qza


In [22]:
# create taxnomic barplot
!qiime taxa barplot \
--i-table table.qza \
--i-taxonomy taxonomy.qza \
--m-metadata-file zooplankton1MetaData.tsv \
--o-visualization taxa-bar-plots

Saved Visualization to: taxa-bar-plots.qzv


In [24]:
# visualising taxonomic barbplot
!qiime tools view taxa-bar-plots.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

## 7. Export QIIME2 artifacts

Export feature-table, taxonomic table and phylogenetic tree from .qza format to 
.biom, .tsv and .nwk formats which could be used by other programs outside QIIME2 (such as R)

In [25]:
# export feature-table to .biom file 
!qiime tools export \
--input-path table.qza \
--output-path exportFile

Exported table.qza as BIOMV210DirFmt to directory exportFile


In [26]:
# export taxonomic table to .tsv file
!qiime tools export \
--input-path taxonomy.qza \
--output-path exportFile

Exported taxonomy.qza as TSVTaxonomyDirectoryFormat to directory exportFile


In [27]:
# export phylogenetic tree to .nwk file
!qiime tools export \
--input-path rooted-tree.qza \
--output-path exportFile

Exported rooted-tree.qza as NewickDirectoryFormat to directory exportFile
